In [1]:
import os
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import json
import infra
import multi_talk as base

# 环境变量名称
API_KEY_NAME = 'TUNNEL_KEY'  # 替换为实际的环境变量名

# 检查是否在Voila环境中运行
IN_VOILA = os.environ.get('VOILA_APP_IP') is not None

# 创建状态变量
key_set_successfully = widgets.Checkbox(value=False, description='密钥已设置', layout=widgets.Layout(display='none'))

# 创建UI组件
status_output = widgets.Output()
with status_output:
    if os.environ.get(API_KEY_NAME):
        display(HTML(f"""
        <div style="background-color:#d4edda; padding:10px; border-radius:5px; margin-bottom:15px;">
            <h3 style="margin-top:0;">✅ API密钥已设置</h3>
            <p style="margin-bottom:0;">环境变量 {API_KEY_NAME} 已配置，应用将使用此密钥。</p>
        </div>
        """))
        key_set_successfully.value = True
    else:
        display(HTML(f"""
        <div style="background-color:#fff3cd; padding:10px; border-radius:5px; margin-bottom:15px;">
            <h3 style="margin-top:0;">⚠️ 需要API密钥</h3>
            <p>请在下方输入您的API密钥以使用完整功能。所有需要API的操作将在设置密钥后生效。</p>
        </div>
        """))

key_input = widgets.Password(
    description='API密钥:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='60%')
)

submit_button = widgets.Button(
    description="设置密钥",
    button_style='primary',
    tooltip='点击设置API密钥',
    icon='check'
)

key_status = widgets.Output()

# 按钮处理函数
def on_submit(b):
    with key_status:
        clear_output()
        if key_input.value.strip():
            # 设置环境变量
            os.environ[API_KEY_NAME] = key_input.value
            display(HTML("""
            <div style="background-color:#d4edda; padding:8px; border-radius:5px;">
                <p style="margin:0;">✅ 密钥已成功设置！任何依赖API的功能现在将正常工作。</p>
            </div>
            """))
            key_set_successfully.value = True
        else:
            display(HTML("""
            <div style="background-color:#f8d7da; padding:8px; border-radius:5px;">
                <p style="margin:0;">❌ 请输入有效的API密钥</p>
            </div>
            """))

submit_button.on_click(on_submit)

# 显示UI组件
display(status_output)
if not os.environ.get(API_KEY_NAME):
    display(key_input)
    display(submit_button)
display(key_status)

# 创建一个检查函数，在需要使用API的地方调用
def check_api_key():
    """
    检查API密钥是否已设置，如果设置了返回True，否则返回False
    """
    if os.environ.get(API_KEY_NAME) or key_set_successfully.value:
        return True
    else:
        with key_status:
            clear_output()
            display(HTML("""
            <div style="background-color:#f8d7da; padding:8px; border-radius:5px;">
                <p style="margin:0;">⚠️ 此操作需要API密钥，请先设置密钥。</p>
            </div>
            """))
        return False

class DialogExtractor:
    def __init__(self):
        # 创建界面元素
        self.title = widgets.HTML("<h1>对话信息提取工具</h1>")
        self.description = widgets.HTML(
            "<p>直接在文本框中粘贴对话内容，系统将自动提取关键信息。</p>"
        )
        
        # 文本输入区域
        self.text_input = widgets.Textarea(
            placeholder='请在此粘贴对话内容...',
            layout=widgets.Layout(width='100%', height='300px')  # 增加文本框高度
        )
        
        # 处理按钮
        self.process_btn = widgets.Button(
            description='提取信息',
            button_style='primary',
            icon='check'
        )
        
        # 进度条
        self.progress = widgets.IntProgress(
            value=0,
            min=0,
            max=4,
            description='处理中:',
            style={'bar_color': '#5DADE2'},
            layout=widgets.Layout(width='50%', visibility='hidden')
        )
        
        # 结果输出区域
        self.output_area = widgets.Output()
        
        # 模型选择
        self.model_dropdown = widgets.Dropdown(
            options=[
                ('gpt', {'name': 'gpt-4o', 'server': 'tunnel', 'model': 'gpt-4o'})
                # 可以添加其他模型选项
            ],
            value={'name': 'gpt-4o', 'server': 'tunnel', 'model': 'gpt-4o'},
            description='选择模型:'
        )
        
        # 布局界面
        self.ui = widgets.VBox([
            self.title,
            self.description,
            self.model_dropdown,
            self.text_input,
            self.process_btn,
            self.progress,
            self.output_area
        ])

        # 注册事件
        self.process_btn.on_click(self._on_process_click)
    
    def _on_process_click(self, b):
        """处理按钮点击事件"""
        if not check_api_key():
            print("未设置api key，请刷新页面重新设置。")
            return  # 如果密钥未设置，直接返回
        
        # 检查是否有内容
        if not self.text_input.value.strip():
            with self.output_area:
                clear_output()
                display(HTML("<div style='color:red'>请输入对话内容！</div>"))
            return
        
        # 显示进度条
        self.progress.layout.visibility = 'visible'
        self.progress.value = 0
        
        with self.output_area:
            clear_output()
            display(HTML("<h3>处理中，请稍候...</h3>"))
        
        try:
            # 提取对话内容
            dialog = self.text_input.value
            self.progress.value = 1
            
            # 构建提示词
            extra_input = f"""对话内容是:
                {dialog}
            """
            
            extra_prompt = """
                你会收到一段用户和老师的对话，用户在对话中，会描述一个孩子的情况，你需要提取以下信息：
                - 儿童基本情况，包括姓名、性别、年龄、年级、是否独生子女、身体健康情况，使用字典格式输出。
                - 父母家庭情况，包括父亲职业、父亲学历、母亲职业、母亲学历、父母婚姻状况、家庭经济情况，使用字典格式输出。
                - 兄弟姐妹的情况，使用列表格式输出，列表中的每个项目是一个字典，包括 关系、年龄 两个key。如果是独生子女，不需要提取这一项。
                - 儿童养育情况：包括出生方式、婴儿期喂养方式、幼儿时期的抚养情况。请根据对话内容，进行一定的概括。使用字典格式输出。
                - 儿童成长情况；幼儿园、小学、初中、高中四个时期儿童的个性特点、学业情况和社交情况。请根据对话内容，进行一定的概括。使用二级字典格式输出，时期是第一级的key。如果孩儿童还未达到某个时期，不需要出现该时期的key。
                - 身心特点：包括认知特点、情绪特点、社会交往特点、兴趣爱好、性别与文化议题。请根据对话内容，进行一定的概括。使用字典格式输出，如果没有提到某项内容，不需要出现对应的key。
                - 拒学发展的情况：孩子的拒学情况是如何发展变化的，请使用列表输出。列表每个项目是对一个事件的拒学情况概括，如"四年级，学习跟不上，经常早上不愿意起床"，不同时间的表现属于不同项目。
                - 当前拒学状态：当前孩子是否能去上学、在学校或家中的学习表现，用几句话概括。
                - 近期状态：当前孩子的饮食、睡眠、购物、运动、社交、生活自理、情绪、自杀自残的情况等。使用字典格式输出，key是某个方面，value是这个方面对应情况的描述。如果某个方面没有提到，则不需要出现对应的key。
                - 重大事件/压力事件：家长提到的一些重要的事件。使用列表输出，每个项目是对一个事件的完整描述，需要提到事件中的具体内容和重要细节。
                - 孩子的态度：孩子自身对学习的态度。用几句话概括。
                - 家长的态度：家长对于孩子不上学的态度。用几句话概括。
                请注意孩子的性别，并使用正确的代词。
                请使用json组织提取到的信息。仅返回json，不要有其他任何内容。
            """
            
            extra_messages = [
                {'role': 'system', 'content': extra_prompt},
                {'role': 'user', 'content': extra_input},
            ]
            
            self.progress.value = 2
            
            # 在调用API处理部分添加以下代码
            try:
                # 设置模型
                services_list = [self.model_dropdown.value]
                
                # 调用API处理
                task = infra.Task(services_list)
                ret = task.assign(extra_messages)
                
            except Exception as e:
                # 使用HTML格式详细打印错误信息
                import traceback
                error_html = f"""
                <div style="background-color:#f8d7da; padding:10px; border-radius:5px; margin:10px 0; color:#721c24; border:1px solid #f5c6cb;">
                    <h3 style="margin-top:0;">API调用失败</h3>
                    <p><strong>错误类型:</strong> {type(e).__name__}</p>
                    <p><strong>错误消息:</strong> {str(e)}</p>
                    <h4>堆栈跟踪:</h4>
                    <pre style="background-color:#f8f9fa; padding:10px; border-radius:3px; overflow:auto; max-height:300px; font-size:12px;">
            {traceback.format_exc()}
                    </pre>
                </div>
                """
                # 显示HTML格式的错误信息
                from IPython.display import display, HTML
                display(HTML(error_html))
                
                # 继续抛出异常，让原有的错误处理逻辑捕获
                raise
            
            self.progress.value = 3
            
            # 解析结果
            if ret and len(ret) > 0:
                result = ret[0]['reply']['content']
                # 去掉可能的```json和```标记
                result = result.strip()
                if result.startswith("```json"):
                    result = result[7:]
                elif result.startswith("```"):
                    result = result[3:]
                
                if result.endswith("```"):
                    result = result[:-3]
                
                result = result.strip()
                try:
                    # 解析JSON
                    json_data = json.loads(result)
                    self._display_results(json_data)
                except json.JSONDecodeError:
                    # 如果不是有效的JSON，直接显示文本
                    with self.output_area:
                        clear_output()
                        display(HTML(f"<pre>{result}</pre>"))
            else:
                with self.output_area:
                    clear_output()
                    display(HTML("<div style='color:red'>处理失败，请重试！</div>"))
            
            self.progress.value = 4
            
        except Exception as e:
            with self.output_area:
                clear_output()
                display(HTML(f"<div style='color:red'>错误: {str(e)}</div>"))
        
        finally:
            # 隐藏进度条
            self.progress.layout.visibility = 'hidden'
    
    def _display_results(self, data):
        """以格式化方式显示结果"""
        with self.output_area:
            clear_output()
            
            # 创建标签页显示结果
            tab = widgets.Tab()
            children = []
            titles = []
            
            # 基本信息
            if "儿童基本情况" in data:
                basic_html = self._dict_to_html(data["儿童基本情况"], "儿童基本情况")
                basic_output = widgets.Output()
                with basic_output:
                    display(HTML(basic_html))
                children.append(basic_output)
                titles.append("基本情况")
            
            # 父母家庭情况
            if "父母家庭情况" in data:
                family_html = self._dict_to_html(data["父母家庭情况"], "父母家庭情况")
                family_output = widgets.Output()
                with family_output:
                    display(HTML(family_html))
                children.append(family_output)
                titles.append("家庭情况")
            
            # 养育情况
            if "儿童养育情况" in data:
                care_html = self._dict_to_html(data["儿童养育情况"], "儿童养育情况")
                care_output = widgets.Output()
                with care_output:
                    display(HTML(care_html))
                children.append(care_output)
                titles.append("养育情况")
            
            # 成长情况
            if "儿童成长情况" in data:
                grow_html = self._dict_to_html(data["儿童成长情况"], "儿童成长情况", nested=True)
                grow_output = widgets.Output()
                with grow_output:
                    display(HTML(grow_html))
                children.append(grow_output)
                titles.append("成长情况")
            
            # 身心特点
            if "身心特点" in data:
                traits_html = self._dict_to_html(data["身心特点"], "身心特点")
                traits_output = widgets.Output()
                with traits_output:
                    display(HTML(traits_html))
                children.append(traits_output)
                titles.append("身心特点")
            
            # 拒学情况
            if "拒学发展的情况" in data:
                refuse_html = self._list_to_html(data["拒学发展的情况"], "拒学发展的情况")
                refuse_output = widgets.Output()
                with refuse_output:
                    display(HTML(refuse_html))
                children.append(refuse_output)
                titles.append("拒学发展")
            
            # 当前拒学状态
            if "当前拒学状态" in data:
                current_html = f"<h3>当前拒学状态</h3><p>{data['当前拒学状态']}</p>"
                current_output = widgets.Output()
                with current_output:
                    display(HTML(current_html))
                children.append(current_output)
                titles.append("当前状态")
            
            # 近期状态
            if "近期状态" in data:
                recent_html = self._dict_to_html(data["近期状态"], "近期状态")
                recent_output = widgets.Output()
                with recent_output:
                    display(HTML(recent_html))
                children.append(recent_output)
                titles.append("近期状态")
            
            # 重大事件
            if "重大事件/压力事件" in data:
                events_html = self._list_to_html(data["重大事件/压力事件"], "重大事件/压力事件")
                events_output = widgets.Output()
                with events_output:
                    display(HTML(events_html))
                children.append(events_output)
                titles.append("重大事件")
            
            # 态度
            attitudes_html = "<h3>态度</h3>"
            if "孩子的态度" in data:
                attitudes_html += f"<h4>孩子的态度</h4><p>{data['孩子的态度']}</p>"
            if "家长的态度" in data:
                attitudes_html += f"<h4>家长的态度</h4><p>{data['家长的态度']}</p>"
            
            attitudes_output = widgets.Output()
            with attitudes_output:
                display(HTML(attitudes_html))
            children.append(attitudes_output)
            titles.append("态度")
            
            # 设置标签页
            tab.children = children
            for i, title in enumerate(titles):
                tab.set_title(i, title)
            
            display(tab)
    
    def _dict_to_html(self, data_dict, title, nested=False):
        """将字典转换为HTML表格"""
        html = f"<h3>{title}</h3>"
        html += "<table style='width:100%; border-collapse: collapse;'>"
        
        if nested:
            # 处理嵌套字典（如儿童成长情况）
            for key, value in data_dict.items():
                html += f"<tr><td colspan='2' style='background-color:#f2f2f2; font-weight:bold; padding:8px; border:1px solid #ddd;'>{key}</td></tr>"
                
                for sub_key, sub_value in value.items():
                    html += f"<tr><td style='width:30%; padding:8px; border:1px solid #ddd;'>{sub_key}</td>"
                    html += f"<td style='padding:8px; border:1px solid #ddd;'>{sub_value}</td></tr>"
        else:
            # 处理普通字典
            for key, value in data_dict.items():
                html += f"<tr><td style='width:30%; padding:8px; border:1px solid #ddd;'>{key}</td>"
                html += f"<td style='padding:8px; border:1px solid #ddd;'>{value}</td></tr>"
        
        html += "</table>"
        return html
    
    def _list_to_html(self, data_list, title):
        """将列表转换为HTML列表"""
        html = f"<h3>{title}</h3>"
        html += "<ul style='margin-left:20px;'>"
        
        for item in data_list:
            html += f"<li style='margin-bottom:8px;'>{item}</li>"
        
        html += "</ul>"
        return html
    
    def display(self):
        """显示界面"""
        display(self.ui)

# 创建并显示工具
extractor = DialogExtractor()
extractor.display()

IndentationError: expected an indented block after function definition on line 163 (908076795.py, line 164)